## optuna 連続値 example notebook

In [1]:
%load_ext lab_black

In [2]:
import optuna
from optuna.samplers import NSGAIISampler
import matplotlib.pyplot as plt

%matplotlib inline

import warnings

warnings.simplefilter("ignore")

In [3]:
# 目的関数の定義。複数の目的変数を戻り値とする
def objective(trial):
    x = trial.suggest_uniform("x", 0, 5)  # 変数xを上下限0~5の範囲で連続値
    y = trial.suggest_uniform("y", 0, 3)  # 変数yを上下限0~3の範囲で連続値
    v0 = 4 * x ** 2 + 4 * y ** 2
    v1 = (x - 5) ** 2 + (y - 5) ** 2
    return v0, v1

In [4]:
%%time
# 最適化の条件設定
study = optuna.multi_objective.create_study(
    directions=["minimize", "minimize"], # "minimize" "maximize"
    sampler=NSGAIISampler(population_size=100,
                          crossover_prob=1.0,
                          mutation_prob=1.0,
                          seed=1)
)
# 最適化の実行
study.optimize(objective, n_trials=200)

[I 2021-07-10 10:02:08,182] A new study created in memory with name: no-name-b8f75542-0953-4148-a86d-9b61b897d4f4
[I 2021-07-10 10:02:08,205] Trial 0 finished with values: [36.06996077131263, 16.556655154434715] with parameters: {'x': 2.08511002351287, 'y': 2.1609734803264744}.
[I 2021-07-10 10:02:08,468] Trial 1 finished with values: [3.2905807492306045, 41.746949267485206] with parameters: {'x': 0.0005718740867244332, 'y': 0.9069977178955193}.
[I 2021-07-10 10:02:08,637] Trial 2 finished with values: [2.4606801279719854, 40.50721764807341] with parameters: {'x': 0.7337794540855652, 'y': 0.2770157843063934}.
[I 2021-07-10 10:02:08,754] Trial 3 finished with values: [7.768126412928175, 32.26219922305707] with parameters: {'x': 0.9313010568883545, 'y': 1.0366821811291431}.
[I 2021-07-10 10:02:08,877] Trial 4 finished with values: [26.194087883052127, 20.54564623912883] with parameters: {'x': 1.9838373711533497, 'y': 1.6164502020100708}.
[I 2021-07-10 10:02:09,013] Trial 5 finished with 

CPU times: user 30.5 s, sys: 1.09 s, total: 31.5 s
Wall time: 33.9 s


In [5]:
# 最適化過程で得た履歴データの取得。get_trials()メソッドを使用
trials = {str(trial.values): trial for trial in study.get_trials()}
trials = list(trials.values())
# グラフにプロットするため、目的変数をリストに格納する
y1_all_list = []
y2_all_list = []
for i, trial in enumerate(trials, start=1):
    y1_all_list.append(trial.values[0])
    y2_all_list.append(trial.values[1])

In [6]:
# パレート解の取得。get_pareto_front_trials()メソッドを使用
trials = {str(trial.values): trial for trial in study.get_pareto_front_trials()}
trials = list(trials.values())
trials.sort(key=lambda t: t.values)
# グラフプロット用にリストで取得。またパレート解の目的変数と説明変数をcsvに保存する
y1_list = []
y2_list = []
with open("pareto_data_real.csv", "w") as f:
    for i, trial in enumerate(trials, start=1):
        if i == 1:
            columns_name_str = "trial_no,y1,y2"
        data_list = []
        data_list.append(trial.number)
        y1_value = trial.values[0]
        y2_value = trial.values[1]
        y1_list.append(y1_value)
        y2_list.append(y2_value)
        data_list.append(y1_value)
        data_list.append(y2_value)
        for key, value in trial.params.items():
            data_list.append(value)
            if i == 1:
                columns_name_str += "," + key
        if i == 1:
            f.write(columns_name_str + "\n")
        data_list = list(map(str, data_list))
        data_list_str = ",".join(data_list)
        f.write(data_list_str + "\n")

# パレート解を図示
plt.rcParams["font.size"] = 16
plt.figure(dpi=120)
plt.title("multiobjective optimization")
plt.xlabel("Y1")
plt.ylabel("Y2")
plt.grid()
plt.scatter(y1_all_list, y2_all_list, c="blue", label="all trials")
plt.scatter(y1_list, y2_list, c="red", label="pareto front")
plt.legend()
plt.tight_layout()
plt.savefig("pareto_graph_real.png")
plt.close()